# October 31 - November 1 night time jet

In [1]:
import sys
sys.path.append('../')
import sosutils
import xarray as xr
import altair as alt
import datetime as dt
alt.data_transformers.disable_max_rows()
import pandas as pd 
import numpy as np
import os

alt.renderers.enable('jupyterlab')

ModuleNotFoundError: No module named 'sosutils'

# User inputs

In [ ]:
download_dir='sosnoqc'
DATE_FORMAT_STR = '%Y%m%d'
datelist = ['20230107', '20230108', '20230109']
start_date = datelist[0]
end_date = datelist[-1]
# end_date = (dt.datetime.today()  - dt.timedelta(days=1)).strftime(DATE_FORMAT_STR)


VARIABLE_NAMES = [ 
    'spd_1m_c', 'dir_1m_c', 'u_1m_c', 'v_1m_c', 'u_w__1m_c', 'u_tc__1m_c', 'v_w__1m_c', 'v_tc__1m_c', 'w_tc__1m_c', 'w_h2o__1m_c',
    'spd_2m_c', 'dir_2m_c', 'u_2m_c', 'v_2m_c', 'u_w__2m_c', 'u_tc__2m_c', 'v_w__2m_c', 'v_tc__2m_c', 'w_tc__2m_c', 'w_h2o__2m_c',
    'spd_3m_c', 'dir_3m_c', 'u_3m_c', 'v_3m_c', 'u_w__3m_c', 'u_tc__3m_c', 'v_w__3m_c', 'v_tc__3m_c', 'w_tc__3m_c', 'w_h2o__3m_c',
    'spd_5m_c', 'dir_5m_c', 'u_5m_c', 'v_5m_c', 'u_w__5m_c', 'u_tc__5m_c', 'v_w__5m_c', 'v_tc__5m_c', 'w_tc__5m_c', 'w_h2o__5m_c',
    'spd_10m_c', 'dir_10m_c', 'u_10m_c', 'v_10m_c', 'u_w__10m_c', 'u_tc__10m_c', 'v_w__10m_c', 'v_tc__10m_c', 'w_tc__10m_c', 'w_h2o__10m_c',
    'spd_15m_c', 'dir_15m_c', 'u_15m_c', 'v_15m_c', 'u_w__15m_c', 'u_tc__15m_c', 'v_w__15m_c', 'v_tc__15m_c', 'w_tc__15m_c', 'w_h2o__15m_c',
    'spd_20m_c', 'dir_20m_c', 'u_20m_c', 'v_20m_c', 'u_w__20m_c', 'u_tc__20m_c', 'v_w__20m_c', 'v_tc__20m_c', 'w_tc__20m_c', 'w_h2o__20m_c',
    'T_1m_c',
    'T_2m_c',
    'T_3m_c',
    'T_4m_c',
    'T_5m_c',
    'T_6m_c',
    'T_7m_c',
    'T_8m_c',
    'T_9m_c',
    'T_10m_c',
    'T_11m_c',
    'T_12m_c',
    'T_13m_c',
    'T_14m_c',
    'T_15m_c',
    'T_16m_c',
    'T_17m_c',
    'T_18m_c',
    'T_19m_c',
    'T_20m_c',
]

# Download SoS data

In [ ]:
datelist

['20230107', '20230108', '20230109']

In [2]:
# if you want to download

ut

datasets = [xr.open_dataset(sosutils.download_sos_data_day(date, download_dir))[VARIABLE_NAMES] for date in datelist]

sos_data = sosutils.merge_datasets_with_different_variables(datasets, dim='time')

NameError: name 'datelist' is not defined

In [ ]:
# # if you don't want to download
# datasets = sorted(os.listdir(download_dir))
# datasets = [xr.open_dataset(os.path.join(download_dir, f)) for f in datasets]
# # sos_data = xr.concat(datasets, dim='time')
# sos_data = sosutils.merge_datasets_with_different_variables(datasets, dim='time')[VARIABLE_NAMES]

In [ ]:
sos_data.coords['time'].min(), sos_data.coords['time'].max()

# Extract usable datasets from the raw SoS data

## Extract first-5-minute-hourly-average data from the raw SoS dataset

In [ ]:
sos_data_1hr = sos_data.where(
    sos_data['time.minute'] < 6, 
    drop=True
).resample(time='60Min').mean() 

sos_data_1hr_df = sos_data_1hr.to_dataframe().reset_index()
sos_data_1hr_df = sos_data_1hr_df.melt(id_vars='time', value_vars=VARIABLE_NAMES)
sos_data_1hr_df['height'] = sos_data_1hr_df['variable'].apply(sosutils.height_from_variable_name)
sos_data_1hr_df['tower'] = sos_data_1hr_df['variable'].apply(sosutils.tower_from_variable_name)
sos_data_1hr_df['measurement'] = sos_data_1hr_df['variable'].apply(sosutils.measurement_from_variable_name)
sos_data_1hr_df['time'] = sos_data_1hr_df['time'] - dt.timedelta(hours=6)

## Extract 5 minute-average data from the raw SoS dataset

In [ ]:
sos_data_df = sos_data.to_dataframe().reset_index().melt(id_vars='time', value_vars=VARIABLE_NAMES)
sos_data_df['height'] = sos_data_df['variable'].apply(sosutils.height_from_variable_name)
sos_data_df['tower'] = sos_data_df['variable'].apply(sosutils.tower_from_variable_name)
sos_data_df['measurement'] = sos_data_df['variable'].apply(sosutils.measurement_from_variable_name)
sos_data_df['time'] = sos_data_df['time'] - dt.timedelta(hours=6)

# Examine wind behavior

In [ ]:
src = sos_data_1hr_df.copy()


In [ ]:
src['day'] = src['time'].dt.day
src['hour'] = src['time'].dt.hour

In [ ]:
src

In [ ]:
src = sos_data_df.copy()
src['date'] = src['time'].dt.date
src['hour'] = src['time'].dt.hour
src

In [ ]:
# get the first date
# get the last date
# make a dataframe containing a FULL night for each 
# modify the first and last entry using the min and max time from the provided dataframe

def create_nighttime_dataframe(start_date, end_date, night_start_hour = 18, night_end_hour = 7):
    start_datetimes = []
    end_datetimes = []
    for date in pd.date_range(start_date, end_date):
        next_day = date + dt.timedelta(days=1)
        start_datetimes.append(dt.datetime(date.year, date.month, date.day, night_start_hour))
        end_datetimes.append(dt.datetime(next_day.year, next_day.month, next_day.day, night_end_hour))
    return pd.DataFrame({'start_date': start_datetimes, 'end_date': end_datetimes})

night_df = create_nighttime_dataframe(sos_data_df.time.min(), sos_data_df.time.max())

night_plot = alt.Chart(night_df).mark_rect(color='grey', opacity=0.2).encode(
    alt.X('start_date:T'),
    alt.X2('end_date')
)

In [ ]:
local_src = sos_data_1hr_df.copy()

# Create additional time columns to make plotting easier
local_src['day'] = local_src['time'].dt.date.apply(lambda date: dt.datetime.combine(date, dt.time()))
local_src['hour'] = local_src['time'].dt.hour
# local_src = local_src[local_src['time'] < dt.datetime(2022, 11, 4)]
# local_src = local_src[local_src['time'].dt.hour.isin([0,3,6,9,12,15,18,21])]
local_src = local_src[local_src['time'].dt.hour.isin([
    0,4,8,12,16,20,24
    # 0,2,4,6,8,10,12,14,16,18,20,22
])]

profile_plot = alt.Chart(local_src).transform_filter(
    alt.datum.measurement == 'wind speed'
).mark_point().encode(
    alt.X(
        'value:Q', 
        sort='-y', 
        title=None, 
        axis=alt.Axis(orient='top', offset=-15)
    ),
    alt.Y('height:Q',  axis=alt.Axis(grid=False)),
    color=alt.Color("height:O", scale=alt.Scale(scheme='viridis'))
).properties(
    width=78,
    height=50
).facet(
    facet=alt.Column(
        'time', 
        # header=alt.Header(format="%m/%d %H:00", formatType='time'),
        header=alt.Header(labelExpr="''"),
        title='Bi-hourly average wind speed profiles (m/s)',
    ),
    columns=24,
   spacing=1
).resolve_axis(
    x='shared'
)


WIND_CHART = alt.Chart(
    sos_data_df
).transform_filter(
    alt.datum.measurement == 'wind direction'
).transform_window(
    rolling_mean = 'mean(value)',
    frame = [-5, 5]
).mark_line().encode(
    alt.X('time:T', axis=alt.Axis(format='%m%d, %H%M')),
    # alt.Y('value:Q'),
    alt.Y(
        'rolling_mean:Q', 
        title = 'Wind Direction (50 minute rolling average)', 
        scale=alt.Scale(domain=[0, 360], nice=False),
        axis=alt.Axis(values=[0, 90, 180, 270, 360]),
        impute=alt.ImputeParams(value=None)
    ),
    color=alt.Color(
        "height:O", 
        scale=alt.Scale(scheme='viridis')
    )
).properties(
    width=1400,
    # height=200
)

In [ ]:
profile_plot

In [ ]:
(WIND_CHART+night_plot).properties(width=2000)

In [ ]:
(
profile_plot & WIND_CHART
).properties(title='Wind direction and vertical profiles over 2 days')
# .configure_axis(grid=False).configure_view(strokeWidth=0)

# Slice data to time period of interest


In [ ]:
# # Original study periods
########################################################################
########################################################################

# 3 hr period of substantial katabatic behavior
###
sos_data_df_slice = sos_data_df[
    (sos_data_df['time'] > dt.datetime(2023, 1, 9, 1))
    &
    (sos_data_df['time'] < dt.datetime(2023, 1, 9, 4))
]

sos_data_1hr_df_slice = sos_data_1hr_df[
    (sos_data_1hr_df['time'] > dt.datetime(2023, 1, 9, 1))
    &
    (sos_data_1hr_df['time'] < dt.datetime(2023, 1, 9, 4))
]

# 3 hr period of substantial anabatic behavior
###
# sos_data_df_slice = sos_data_df[
#     (sos_data_df['time'] > dt.datetime(2022, 11, 1, 10, 30))
#     &
#     (sos_data_df['time'] < dt.datetime(2022, 11, 1, 13, 30))
# ]

# sos_data_1hr_df_slice = sos_data_1hr_df[
#     (sos_data_1hr_df['time'] > dt.datetime(2022, 11, 1, 10, 30))
#     &
#     (sos_data_1hr_df['time'] < dt.datetime(2022, 11, 1, 13, 30))
# ]




# # Dates aligned with good Doppler LIDAR reads
########################################################################
########################################################################

# 3 hr period of substantial katabatic behavior
###
# sos_data_df_slice = sos_data_df[
#     (sos_data_df['time'] > dt.datetime(2022, 11, 1, 1))
#     &
#     (sos_data_df['time'] < dt.datetime(2022, 11, 1, 4))
# ]

# sos_data_1hr_df_slice = sos_data_1hr_df[
#     (sos_data_1hr_df['time'] > dt.datetime(2022, 11, 1, 1))
#     &
#     (sos_data_1hr_df['time'] < dt.datetime(2022, 11, 1, 4))
# ]

# 3 hr period of substantial anabatic behavior
###
# sos_data_df_slice = sos_data_df[
#     (sos_data_df['time'] > dt.datetime(2022, 10, 31, 12))
#     &
#     (sos_data_df['time'] < dt.datetime(2022, 10, 31, 14))
# ]

# sos_data_1hr_df_slice = sos_data_1hr_df[
#     (sos_data_1hr_df['time'] > dt.datetime(2022, 10, 31, 12))
#     &
#     (sos_data_1hr_df['time'] < dt.datetime(2022, 10, 31, 14))
# ]





# ALL OF IT
########################################################################
########################################################################

# 3 hr period of substantial katabatic behavior
###
# sos_data_df_slice = sos_data_df[
#     (sos_data_df['time'] > dt.datetime(2022, 10, 31, 12))
#     &
#     (sos_data_df['time'] < dt.datetime(2022, 11, 1, 19))
# ]

# sos_data_1hr_df_slice = sos_data_1hr_df[
#     (sos_data_1hr_df['time'] > dt.datetime(2022, 10, 31, 12))
#     &
#     (sos_data_1hr_df['time'] < dt.datetime(2022, 11, 1, 19))
# ]

In [ ]:
dl_windprofile_df = dl_windprofile_data.to_dataframe().reset_index()
dl_windprofile_df_slice = dl_windprofile_df[
    (dl_windprofile_df['time'] > dt.datetime(2022, 10, 31, 18))
    &
    (dl_windprofile_df['time'] < dt.datetime(2022, 11, 1, 6))
][['time', 'bound', 'height', 'wind_speed', 'wind_speed_error',
       'wind_direction', 'wind_direction_error']]

## Look at DL and tower winds during the time period

In [ ]:
alt.Chart(
    dl_windprofile_df_slice
).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    # alt.Y(
    #     'rolling_mean:Q', 
    #     title = 'Wind Direction (50 minute rolling average', 
    #     scale=alt.Scale(domain=[0, 360], nice=False),
    #     axis=alt.Axis(values=[0, 90, 180, 270, 360])

    # ),
    color=alt.Color(
        "height:Q", 
        scale=alt.Scale(scheme='viridis')
    )
)

In [ ]:
alt.Chart(
    sos_data_df_slice
).transform_filter(
    alt.datum.measurement == 'wind direction'
).transform_window(
    rolling_mean = 'mean(value)',
    frame = [-5, 5]
).mark_line().encode(
    alt.X('time:T'),
    # alt.Y('value:Q'),
    alt.Y(
        'rolling_mean:Q', 
        title = 'Wind Direction (50 minute rolling average', 
        scale=alt.Scale(domain=[0, 360], nice=False),
        axis=alt.Axis(values=[0, 90, 180, 270, 360])
    ),
    color=alt.Color(
        "height:O", 
        scale=alt.Scale(scheme='viridis')
    )
).properties(
    width=1000,
    # height=200
)
# .configure_axis(grid=False).configure_view(strokeWidth=0)

In [ ]:
local_src = sos_data_1hr_df_slice.copy()

# Create additional time columns to make plotting easier
local_src['day'] = local_src['time'].dt.date.apply(lambda date: dt.datetime.combine(date, dt.time()))
local_src['hour'] = local_src['time'].dt.hour
local_src = local_src[local_src['time'] < dt.datetime(2022, 11, 4)]
# local_src = local_src[local_src['time'].dt.hour.isin([0,3,6,9,12,15,18,21])]

profile_plot = alt.Chart(local_src).transform_filter(
    alt.datum.measurement == 'wind speed'
).mark_point().encode(
    alt.X(
        'value:Q', 
        sort='-y', 
        title=None, 
        axis=alt.Axis(orient='top', offset=-15)
    ),
    alt.Y('height:Q',  axis=alt.Axis(grid=False)),
    color=alt.Color("height:O", scale=alt.Scale(scheme='viridis'))
).properties(
    width=83.4,
    height=50
).facet(
    facet=alt.Column(
        'time', 
        header=alt.Header(format="%H:00", formatType='time'),
        # header=alt.Header(labelExpr="''"),
        title='Hourly average wind speed profiles (m/s)',
    ),
    columns=24,
   spacing=1
).resolve_axis(
    x='shared'
)

(
profile_plot & \
alt.Chart(
    sos_data_df_slice
).transform_filter(
    alt.datum.measurement == 'wind direction'
).transform_window(
    rolling_mean = 'mean(value)',
    frame = [-5, 5]
).mark_line().encode(
    alt.X('time:T'),
    # alt.Y('value:Q'),
    alt.Y(
        'rolling_mean:Q', 
        title = 'Wind Direction (50 minute rolling average', 
        scale=alt.Scale(domain=[0, 360], nice=False),
        axis=alt.Axis(values=[0, 90, 180, 270, 360])

    ),
    color=alt.Color(
        "height:O", 
        scale=alt.Scale(scheme='viridis')
    )
).properties(
    width=1000,
    # height=200
)).properties(title='Wind direction and vertical profiles over 2 days')
# .configure_axis(grid=False).configure_view(strokeWidth=0)

In [ ]:
local_src = sos_data_df_slice.set_index('time').groupby([pd.Grouper(freq='15Min'), 'tower', 'height', 'measurement']).mean().reset_index()

# Create additional time columns to make plotting easier
local_src['day'] = local_src['time'].dt.date.apply(lambda date: dt.datetime.combine(date, dt.time()))
local_src['hour'] = local_src['time'].dt.hour
local_src = local_src[local_src['time'] < dt.datetime(2022, 11, 4)]
# local_src = local_src[local_src['time'].dt.hour.isin([0,3,6,9,12,15,18,21])]

profile_plot = alt.Chart(local_src).transform_filter(
    alt.datum.measurement == 'wind speed'
).mark_point().encode(
    alt.X(
        'value:Q', 
        sort='-y', 
        title=None, 
        axis=alt.Axis(orient='top', offset=-30)
    ),
    alt.Y('height:Q',  axis=alt.Axis(grid=False)),
    color=alt.Color("height:O", scale=alt.Scale(scheme='viridis'))
).properties(
    width=81,
    height=50
).facet(
    facet=alt.Column(
        'time', 
        header=alt.Header(format="%H:%M", formatType='time', labelPadding=30, titlePadding=30),
        # header=alt.Header(labelExpr="''"),
        title='15 minute average wind speed profiles (m/s)',
    ),
    columns=24,
   spacing=1
).resolve_axis(
    x='shared'
)

(
profile_plot & \
alt.Chart(
    sos_data_df_slice
).transform_filter(
    alt.datum.measurement == 'wind direction'
).transform_window(
    rolling_mean = 'mean(value)',
    frame = [-5, 5]
).mark_line().encode(
    alt.X('time:T'),
    # alt.Y('value:Q'),
    alt.Y(
        'rolling_mean:Q', 
        title = 'Wind Direction (50 minute rolling average', 
        scale=alt.Scale(domain=[0, 360], nice=False),
        axis=alt.Axis(values=[0, 90, 180, 270, 360])

    ),
    color=alt.Color(
        "height:O", 
        scale=alt.Scale(scheme='viridis')
    )
).properties(
    width=1000,
    # height=200
)).properties(title='Wind direction and vertical profiles over 2 days')
# .configure_axis(grid=False).configure_view(strokeWidth=0)

# Convert sonic data to upstream coordinates (ignoring rotation in the z-normal axis, i.e. not adjusting for sloping topography)

This requires averaging over a period of near-constant wind direction. Let's study the second night, 6pm Oct 31 to 6am Nov 1.

## Look at U and V

In [ ]:


horzline = alt.Chart().mark_rule().encode(
    y='a:Q'
)

og_plot = alt.Chart(
).mark_point().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color("height:O", scale=alt.Scale(scheme='viridis')),
    # alt.Facet('measurement', columns=1)
).properties(
    width=300,
    height=100
).transform_filter(
    alt.FieldOneOfPredicate('measurement', ['u', 'v'])
)

alt.layer(
    og_plot,
    horzline, 
    data = sos_data_df_slice.dropna()
).transform_filter(
    alt.FieldOneOfPredicate('measurement', ['u', 'v'])
).transform_calculate(
        a="0"
).facet('measurement', columns=1)

## "1. Calculate mean U and V (in sonic coords)



Find mean U and V for a given period, and use equations from EOL (https://www.eol.ucar.edu/node/1953)

The U axis (Ustream) of streamwise coordinates is defined to be the mean wind direction. To rotate wind vectors to streamwise coordinates, first determine the the average wind vector,Uav, Vav, in the same coordinate system as the data to be rotated, which could be instrument or geographic coordinates. The rotation angle is the angle of this wind vector from the U axis, measured positive counter-clockwise.

1. Find the average wind vector:
    <U, V>

In [ ]:
u_avg = sos_data_df_slice[sos_data_df_slice['measurement'] == 'u'].groupby(['tower', 'height'])['value'].mean()
v_avg = sos_data_df_slice[sos_data_df_slice['measurement'] == 'v'].groupby(['tower', 'height'])['value'].mean()
u_avg, v_avg

## "2. Compute the adjustment angles angles

D = atan2(Vav,Uav) * DperR

Ustream =  U * cos(D*RperD) + V * sin(D*RperD)

Vstream = -U * sin(D*RperD) + V * cos(D*RperD)

As expected, if U=Uav and V=Vav then Ustream = Spd, and Vstream = 0.

In [ ]:
D = np.arctan2(v_avg, u_avg)

In [ ]:
D

In [ ]:
sos_data_df_slice_wide = sos_data_df_slice.pivot_table(index=['time','tower','height'], values='value', columns='measurement').reset_index()

In [ ]:
sos_data_df_slice_wide.apply(
    lambda row: row['u']*np.cos(
        # D.loc[row['tower'], row['height']]
        D.to_dict().get((row['tower'], row['height']), np.nan)
    ) + row['v']*np.sin(
        # D.loc[row['tower'], row['height']]
        D.to_dict().get((row['tower'], row['height']), np.nan)
    ),
    axis=1
)

In [ ]:
# Ustream =  U * cos(D*RperD) + V * sin(D*RperD)

# Vstream = -U * sin(D*RperD) + V * cos(D*RperD)

# convert u velocity
sos_data_df_slice_wide['u'] = sos_data_df_slice_wide.apply(
    lambda row: row['u']*np.cos(D.to_dict().get((row['tower'], row['height']), np.nan)) + row['v']*np.sin(D.to_dict().get((row['tower'], row['height']), np.nan)),
    axis=1
)

# # convert v velocity
sos_data_df_slice_wide['v'] = sos_data_df_slice_wide.apply(
    lambda row: - row['u']*np.sin(D.to_dict().get((row['tower'], row['height']), np.nan)) + row['v']*np.cos(D.to_dict().get((row['tower'], row['height']), np.nan)),
    axis=1
)

# # convert u_tc_ and u_w_ same as we convert u - IS THIS CORRECT?
sos_data_df_slice_wide['u_tc_'] = sos_data_df_slice_wide.apply(
    lambda row: row['u_tc_']*np.cos(D.to_dict().get((row['tower'], row['height']), np.nan)) + row['v_tc_']*np.sin(D.to_dict().get((row['tower'], row['height']), np.nan)),
    axis=1
)

sos_data_df_slice_wide['u_w_'] = sos_data_df_slice_wide.apply(
    lambda row: row['u_w_']*np.cos(D.to_dict().get((row['tower'], row['height']), np.nan)) + row['v_w_']*np.sin(D.to_dict().get((row['tower'], row['height']), np.nan)),
    axis=1
)

# # convert v_tc_ and v_w_  same as we convert v - IS THIS CORRECT?
sos_data_df_slice_wide['v_tc_'] = sos_data_df_slice_wide.apply(
    lambda row: - row['u_tc_']*np.sin(D.to_dict().get((row['tower'], row['height']), np.nan)) + row['v_tc_']*np.cos(D.to_dict().get((row['tower'], row['height']), np.nan)),
    axis=1
)

sos_data_df_slice_wide['w_tc_'] = sos_data_df_slice_wide.apply(
    lambda row: - row['u_w_']*np.sin(D.to_dict().get((row['tower'], row['height']), np.nan)) + row['v_w_']*np.cos(D.to_dict().get((row['tower'], row['height']), np.nan)),
    axis=1
)

In [ ]:
pd.set_option('display.max_rows', 500)

## Create rotated dataset in tidy format

In [ ]:
sos_data_slice_rot = sos_data_df_slice_wide.melt(id_vars=['time', 'tower', 'height'], value_vars=sos_data_df_slice['measurement'].unique())

In [ ]:


horzline = alt.Chart().mark_rule().encode(
    y='a:Q'
)

og_plot = alt.Chart(
).mark_point().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color("height:O", scale=alt.Scale(scheme='viridis')),
    # alt.Facet('measurement', columns=1)
).properties(
    width=300,
    height=100
).transform_filter(
    alt.FieldOneOfPredicate('measurement', ['u', 'v'])
)

alt.layer(
    og_plot,
    horzline, 
    data = sos_data_slice_rot.dropna()
).transform_filter(
    alt.FieldOneOfPredicate('measurement', ['u', 'v'])
).transform_calculate(
        a="0"
).facet('measurement', columns=1)

In [ ]:
def grid_plot_with_line(data, columns, xlabel=None):
    
    og_plot = alt.Chart().mark_line(point=False).encode(
        alt.X('value:Q', sort='y', title=xlabel),
        alt.Y('height:Q', title='Height (m)'),
        # alt.Color("height:O", scale=alt.Scale(scheme='viridis')),
    ).properties(
        width=100,
        height=100
    )

    vertline = alt.Chart().mark_rule().encode(
        x='a:Q'
    )

    return alt.layer(
            og_plot, vertline,
            data=data
        ).transform_filter(
           alt.FieldOneOfPredicate('measurement', columns)
        ).transform_calculate(
            a="0"
        ).facet(
            column= alt.Column('time', header=alt.Header(format="%m/%d %H:%M", formatType='time')),
            row = alt.Row('measurement', sort=columns)
        ).resolve_scale(
            y='shared', 
        # x='shared'
        )

In [ ]:
src = sos_data_slice_rot.set_index('time').groupby([pd.Grouper(freq='60Min'), 'height', 'tower', 'measurement']).mean().reset_index()

# drop any nans because they mess up plotting continuously
src = src.dropna(subset = 'value')

(
    grid_plot_with_line(src, ['T'], xlabel='Temperature (C˚)') & 
    # grid_plot_with_line(src, ['wind speed', 'u']) &
    grid_plot_with_line(src, ['u'], xlabel='Wind speed (m/s)') &
    grid_plot_with_line(src, ['u_w_'], xlabel=["Vertical momentum", "flux (m^2/s^2)"]) & 
    grid_plot_with_line(src, ['w_tc_'], xlabel=["Vertical heat", "flux (m^2 ˚C/s)"]) &
    grid_plot_with_line(src, ['w_h2o_'], xlabel=["Vertical moisture", "flux (g/m^2/s)"])    
).configure_axis(grid=False)

In [ ]:
def grid_plot_with_line_wxlim(data, columns, xlabel=None, ylabel = None, xlim=None):
    
    og_plot = alt.Chart().mark_line(point=False).encode(
        alt.X('value:Q', sort='y', title=xlabel, scale=alt.Scale(domain=xlim)),
        alt.Y('height:Q', title=ylabel),
        # alt.Color("height:O", scale=alt.Scale(scheme='viridis')),
    ).properties(
        width=100,
        height=100
    )

    vertline = alt.Chart().mark_rule().encode(
        x='a:Q'
    )

    return alt.layer(
            og_plot, vertline,
            data=data
        ).transform_filter(
           alt.FieldOneOfPredicate('measurement', columns)
        ).transform_calculate(
            a="0"
        ).resolve_scale(
            y='shared', 
        # x='shared'
        )

In [ ]:
src = sos_data_slice_rot.set_index('time').groupby([pd.Grouper(freq='60Min'), 'height', 'tower', 'measurement']).mean().reset_index()
# filter dates,
#  drop any nans because they mess up plotting continuously
src1 = src[(src['time'].dt.day == 31) & (src['time'].dt.hour == 13)].dropna(subset = 'value')

src2 = src[(src['time'].dt.day == 31) & (src['time'].dt.hour == 18)].dropna(subset = 'value')

src3 = src[(src['time'].dt.day == 1) & (src['time'].dt.hour == 2)].dropna(subset = 'value')

src4 = src[(src['time'].dt.day == 1) & (src['time'].dt.hour == 18)].dropna(subset = 'value')


src = src

(
    (
        grid_plot_with_line_wxlim(src1, ['wind speed'], xlabel='', xlim=[0, 5], ylabel = 'Height (m)') |
        grid_plot_with_line_wxlim(src1, ['w_h2o_'], xlabel="", xlim=[-0.01, 0.01]) |
        grid_plot_with_line_wxlim(src1, ['T'], xlabel="", xlim=[-7, 7])
    ) 
    &
    (
        grid_plot_with_line_wxlim(src2, ['wind speed'], xlabel='', xlim=[0, 5], ylabel = 'Height (m)') |
        grid_plot_with_line_wxlim(src2, ['w_h2o_'], xlabel="", xlim=[-0.01, 0.01]) |
        grid_plot_with_line_wxlim(src2, ['T'], xlabel="", xlim=[-7, 7])
    ) 
    &
    (
        grid_plot_with_line_wxlim(src3, ['wind speed'], xlabel='', xlim=[0, 5], ylabel = 'Height (m)') |
        grid_plot_with_line_wxlim(src3, ['w_h2o_'], xlabel="", xlim=[-0.01, 0.01]) |
        grid_plot_with_line_wxlim(src3, ['T'], xlabel="", xlim=[-7, 7])
    )
    &
    (
        grid_plot_with_line_wxlim(src4, ['wind speed'], xlabel='Wind speed (m/s)', xlim=[0, 5], ylabel = 'Height (m)') |
        grid_plot_with_line_wxlim(src4, ['w_h2o_'], xlabel=["Vertical moisture", "flux (g/m^2/s)"], xlim=[-0.01, 0.01]) |
        grid_plot_with_line_wxlim(src4, ['T'], xlabel='Temperature (C˚)', xlim=[-7, 7])
    )
).configure_axis(grid=False)